In [1]:
import $ivy.`org.apache.spark:spark-sql_2.13:3.3.2`
import org.apache.spark.sql._
import org.apache.spark.sql.functions._

val spark = SparkSession.builder()
  .appName("SparkTest")
  .master("local[*]")
  .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")
  
// http://localhost:4040/jobs/
println("Spark is ready!")

SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties


Spark is ready!


import $ivy.$                                      

import org.apache.spark.sql._

import org.apache.spark.sql.functions._


spark: SparkSession = org.apache.spark.sql.SparkSession@44a0a51f

In [2]:
import spark.implicits._
import java.sql.Date

case class Delivery(
  delivery_id: Int,
  customer_id: Int,
  order_date: Date,
  customer_pref_delivery_date: Date
)

val deliveryData: Seq[(Int, Int, Date, Date)] = Seq(
  (1, 1, Date.valueOf("2019-08-01"), Date.valueOf("2019-08-02")),
  (2, 2, Date.valueOf("2019-08-02"), Date.valueOf("2019-08-02")),
  (3, 1, Date.valueOf("2019-08-11"), Date.valueOf("2019-08-12")),
  (4, 3, Date.valueOf("2019-08-24"), Date.valueOf("2019-08-24")),
  (5, 3, Date.valueOf("2019-08-21"), Date.valueOf("2019-08-22")),
  (6, 2, Date.valueOf("2019-08-11"), Date.valueOf("2019-08-13")),
  (7, 4, Date.valueOf("2019-08-09"), Date.valueOf("2019-08-09"))
)

val deliveryDS = deliveryData.toDF(
  "delivery_id",
  "customer_id",
  "order_date",
  "customer_pref_delivery_date"
).as[Delivery]

import spark.implicits._

import java.sql.Date


defined class Delivery
deliveryData: Seq[(Int, Int, Date, Date)] = List(
  (1, 1, 2019-08-01, 2019-08-02),
  (2, 2, 2019-08-02, 2019-08-02),
  (3, 1, 2019-08-11, 2019-08-12),
  (4, 3, 2019-08-24, 2019-08-24),
  (5, 3, 2019-08-21, 2019-08-22),
  (6, 2, 2019-08-11, 2019-08-13),
  (7, 4, 2019-08-09, 2019-08-09)
)
deliveryDS: Dataset[Delivery] = [delivery_id: int, customer_id: int ... 2 more fields]

If the customer's preferred delivery date is the same as the order date, then the order is called immediate; otherwise, it is called scheduled.

The first order of a customer is the order with the earliest order date that the customer made. It is guaranteed that a customer has precisely one first order.

Write a solution to find the percentage of immediate orders in the first orders of all customers, rounded to 2 decimal places.

The result format is in the following example.

| immediate_percentage |
|----------------------|
| 50.00                |

In [3]:
deliveryDS.show()

+-----------+-----------+----------+---------------------------+
|delivery_id|customer_id|order_date|customer_pref_delivery_date|
+-----------+-----------+----------+---------------------------+
|          1|          1|2019-08-01|                 2019-08-02|
|          2|          2|2019-08-02|                 2019-08-02|
|          3|          1|2019-08-11|                 2019-08-12|
|          4|          3|2019-08-24|                 2019-08-24|
|          5|          3|2019-08-21|                 2019-08-22|
|          6|          2|2019-08-11|                 2019-08-13|
|          7|          4|2019-08-09|                 2019-08-09|
+-----------+-----------+----------+---------------------------+



In [4]:
deliveryDS.groupBy("customer_id")
    .agg(min(col("order_date")).as("order_date"))
    .join(deliveryDS, Seq("customer_id", "order_date"))
    .withColumn("enable", when(col("order_date") === col("customer_pref_delivery_date"), 1).otherwise(0))
    .select(col("enable"))
    .agg(round(sum(col("enable")) * 100 / count(lit(1)), 2).as("immediate_percentage"))
    .show()

+--------------------+
|immediate_percentage|
+--------------------+
|                50.0|
+--------------------+



In [5]:
spark.stop